<a href="https://colab.research.google.com/github/KATHIRSKAT/ML/blob/master/pytorchCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
import torchvision.transforms as transforms

In [58]:
class CNN(nn.Module):
    def __init__(self, in_channels=1, num_classes=10):
        super(CNN, self).__init__()
        # Define the layers
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.pool = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.fc1 = nn.Linear(16 * 7 * 7, num_classes)

    def forward(self, x):
        # Forward pass through the network
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.reshape(x.shape[0], -1)  # Flatten the tensor
        x = self.fc1(x)  # Fully connected layer

        return x


In [60]:
import torch.cuda
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [61]:
in_channels = 1
num_classes = 10
epochs = 5
lr = 0.0001
batch_size = 64

In [62]:
traindata = MNIST(root = "dataset/",train = True,transform = transforms.ToTensor(),download=True)
train_batches = DataLoader(dataset  = traindata,batch_size= batch_size , shuffle = True)
testdata = MNIST(root = "dataset/",train = False,transform = transforms.ToTensor(),download=True)
test_batches = DataLoader(dataset  = traindata,batch_size= batch_size , shuffle = True)

In [63]:
model = CNN().to(device)

In [64]:
creterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=lr)

In [66]:
for i in range(epochs):
  for batch_idx ,(data,targets) in enumerate(train_batches):
    data = data.to(device=device)
    targets = targets.to(device=device)
    scores= model(data)
    loss = creterion(scores,targets)

    optimizer.zero_grad()
    loss.backward()

    optimizer.step()

In [51]:
def check_accuracy(loader_data,model):
  num_sam = 0
  num_crt = 0

  model.eval()
  with torch.no_grad():
    for x,y in (loader_data):
      x = x.to(device)
      y = y.to(device)

      scores = model(x)
      _, predict = scores.max(1)
      num_crt += (predict == y).sum()
      num_sam += len(y)

    model.train()
    return num_crt/num_sam

In [67]:
print(check_accuracy(test_batches,model))

tensor(0.9669)
